# Mode Selection
Set finetuning = 1 if you want to finetune, or set finetuning = 0 if you want to just load the saved model to get inference.

In [1]:
finetuning = 1
load_and_run = not finetuning

# Load Data

In [2]:
import pandas as pd

# data = pd.read_csv('/kaggle/input/gb-all-necessary/train_test_sarcasm_data_mod.csv')
data = pd.read_csv('train_test_sarcasm_data_mod.csv')
data

,male,female,_original_label,test
0,ভাই মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,আফা মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,1,0
1,ভাই মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,বোন মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,1,0
2,ভাই মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,ভাবী মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে প...,1,1
3,নিশ্চয় সাংবাদিককে কোনো কিছু দেওয়ার প্রতিশ্রুত...,নিশ্চয় মহিলা সাংবাদিককে কোনো কিছু দেওয়ার প্রতি...,1,0
4,দুইদিন বৃষ্টিতে একটু সুস্থ হইছে । ভাদ্র মাসের ...,দুইদিন বৃষ্টিতে একটু সুস্থ হইছে । ভাদ্র মাসে...,1,0
...,...,...,...,...
9245,ডিজিটাল মিটার চালু হইছে তো ভাই অনেক আগে থেকে ত...,ডিজিটাল মিটার চালু হইছে তো আফা অনেক আগে থেকে ত...,0,1
9246,ডিজিটাল মিটার চালু হইছে তো ভাই অনেক আগে থেকে ত...,ডিজিটাল মিটার চালু হইছে তো বোন অনেক আগে থেকে ত...,0,1
9247,ডিজিটাল মিটার চালু হইছে তো ভাই অনেক আগে থেকে ত...,ডিজিটাল মিটার চালু হইছে তো ভাবী অনেক আগে থেকে ...,0,0
9248,আমরা ঢাকাবাসীর গৌরবান্বিত যে টাইগার সম্রাট সাক...,আমরা ঢাকাবাসীর গৌরবান্বিত যে টাইগার সম্রাজ্ঞী ...,0,0


# Train, Test, Validation Split
We are going to consider the data as test data if the prediction between original sentence and NER converted sentence become unequal.

In [3]:
temp = data[data['test']!= 1]
test = data[data['test']== 1]
# validation = temp.groupby('_original_label', group_keys=False).apply(lambda x: x.sample(frac=0.2, random_state=1234))
validation = temp.groupby('_original_label', group_keys=False).apply(lambda x: x.sample(frac=0.02, random_state=1234))
train = temp.drop(validation.index)
print('Train class distribution: ', train._original_label.value_counts())
diff  = max(train._original_label.value_counts()) - min(train._original_label.value_counts())
print('Difference (Change carefully): ', diff)
ex = train[train._original_label==0].sample(n = diff, random_state = 1234)
train = pd.concat([train, ex])
train = train.reset_index(drop=True)
print('Train class distribution: ', train._original_label.value_counts())

Train class distribution:  _original_label
1    4112
0    3140
Name: count, dtype: int64
Difference (Change carefully):  972
Train class distribution:  _original_label
1    4112
0    4112
Name: count, dtype: int64


C:\Users\austg\AppData\Local\Temp\ipykernel_9368\3271618259.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  validation = temp.groupby('_original_label', group_keys=False).apply(lambda x: x.sample(frac=0.02, random_state=1234))


In [4]:
train

,male,female,_original_label,test
0,ভাই মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,আফা মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,1,0
1,ভাই মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,বোন মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,1,0
2,নিশ্চয় সাংবাদিককে কোনো কিছু দেওয়ার প্রতিশ্রুত...,নিশ্চয় মহিলা সাংবাদিককে কোনো কিছু দেওয়ার প্রতি...,1,0
3,দুইদিন বৃষ্টিতে একটু সুস্থ হইছে । ভাদ্র মাসের ...,দুইদিন বৃষ্টিতে একটু সুস্থ হইছে । ভাদ্র মাসে...,1,0
4,দুইদিন বৃষ্টিতে একটু সুস্থ হইছে । ভাদ্র মাসের ...,দুইদিন বৃষ্টিতে একটু সুস্থ হইছে । ভাদ্র মাসে...,1,0
...,...,...,...,...
8219,বাবার উত্তম সন্ত ান । আল্লাহ নেক হায়াৎ বাড়ি...,মায়ের উত্তম সন্তান । আল্লাহ নেক হায়াৎ বাড়িয়ে দ...,0,0
8220,ভাই একেবারে সুস্থ্য হয়ে তবেই খেলতে নামবেন । বা...,বোন একেবারে সুস্থ্য হয়ে তবেই খেলতে নামবেন । ...,0,0
8221,নায়ক হইছে অথচ তার কাপড় খোলার অভ্যাসটা যায় নি ...,নায়িকা হইছে অথচ তার কাপড় খোলার অভ্যাসটা যায় নি...,0,0
8222,সাকিব ভাই তুমি সবচেয়ে একটা ভাল কাজ করছ বিপদের ...,দময়ন্তী ভাবী তুমি সবচেয়ে একটা ভাল কাজ করছ বিপ...,0,0


In [5]:
validation

,male,female,_original_label,test
5429,যাঁর সঙ্গে জড়িয়ে আছে নাড়ির টান জীবনের প্রথম ...,যাঁর সঙ্গে জড়িয়ে আছে নাড়ির টান জীবনের প্রথম ...,0,0
9074,আমি স্বপ্ন দেখি অসহায় মানুষ ও বন্যপ্রাণীর জন্য...,আমি স্বপ্ন দেখি অসহায়া মানুষ ও বন্যপ্রাণীর জন্...,0,0
8031,কাবিননামা নিয়ে কোন প্রভাব পড়বে? ভাই জানতে ইচ্ছ...,কাবিননামা নিয়ে কোন প্রভাব পড়বে? ভাবী জানতে ইচ্...,0,0
8454,বুড়া বয়সে দুই কোটির জন্য আশা করে বসে না থেকে ম...,বুড়ি বয়সে দুই কোটির জন্য আশা করে বসে না থেকে ম...,0,0
8776,জামালপুর হোম ডেলিভারি হবে ভাই,জামালপুর হোম ডেলিভারি হবে আফা,0,0
...,...,...,...,...
2719,সারারাত জেগে ছিলাম এই ছবি দেখার আশায়কবে জ্বিন ...,সারারাত জেগে ছিলাম এই ছবি দেখার আশায়কবে পরী ছব...,1,0
2421,ক্রিকেটার হইলে এত সুন্দর বউ পাওয়া যায় জানলে পড়...,ক্রিকেটার হইলে এত সুন্দর জামাই পাওয়া যায় জানলে...,1,0
42,ভাই ইন্ডিয়া তে গেলে সবাই মোটা হয়ে যায় আমিও চিক...,বোন ইন্ডিয়া তে গেলে সবাই মোটা হয়ে যায় আমিও চিক...,1,0
2986,যতসব পাগল ছাগল টিম মেনেজমেন্ট রেখেছে বিসিবি ।,যতসব পাগলী ছাগল টিম মেনেজমেন্ট রেখেছে বিসিবি ।,1,0


In [4]:
test

,male,female,_original_label,test
2,ভাই মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,ভাবী মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে প...,1,1
11,দে বাবা দে বলি দে । জবা গলা জোড়াতে ও পারে,দে মা দে বলি দে । জবা গলা জোড়াতে ও পারে,1,1
20,আপনি কি জন্য আপনার বাপের বয়সী বেডার লগে ক্লাবে...,আপনি কি জন্য আপনার মায়ের বয়সী বেডির লগে ক্লাবে...,1,1
29,বন্ধু মানুষ আসলে বাঙ্গির মতো ।,বন্ধুপত্নী মানুষ আসলে বাঙ্গির মতো ।,1,1
34,আয় হায় ! কি ঢস টা দিলো রে আতিক ভাই !,আয় হায় ! কি ঢস টা দিলো রে বিনিতা বোন !,1,1
...,...,...,...,...
9242,সমানভাবে ভাগ করায় সাংবাদিক সাহেবও খুশি,সমানভাবে ভাগ করায় মহিলা সাংবাদিক সাহেবও খুশি,0,1
9243,ত্রিশ দিন রমজানের পর আজকে শয়তান ছাড়া পেয়েছে ৷,ত্রিশ দিন রমজানের পর আজকে শয়তানি ছাড়া পেয়েছে ৷,0,1
9245,ডিজিটাল মিটার চালু হইছে তো ভাই অনেক আগে থেকে ত...,ডিজিটাল মিটার চালু হইছে তো আফা অনেক আগে থেকে ত...,0,1
9246,ডিজিটাল মিটার চালু হইছে তো ভাই অনেক আগে থেকে ত...,ডিজিটাল মিটার চালু হইছে তো বোন অনেক আগে থেকে ত...,0,1


# Minimizing KL Divergence

## Load Pretrained Model

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transform
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForMaskedLM, BertConfig
import pandas as pd
import numpy as np
import transformers
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from tqdm import tqdm
from tqdm import tqdm
import torch
import torch.optim as optim
import torch.nn.functional as F
from collections import OrderedDict
from datetime import datetime
from transformers import get_linear_schedule_with_warmup

def load_transformer_based_model(model_path):
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForMaskedLM.from_pretrained(model_path ,output_hidden_states=True)
    return tokenizer, model
model_path = "raquiba/sarcasm-detection-BanglaSARC"
tokenizer, b_model = load_transformer_based_model(model_path)
print('Total Pretrained tokens: ',len(tokenizer.get_vocab()))
tokenizer.add_tokens(['<Name>' ,'<Gender>'])
b_model.resize_token_embeddings(len(tokenizer))
print('Total number of tokens after adding the new ones: ',len(tokenizer.get_vocab()))

c:\Users\austg\.conda\envs\Temp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of ElectraForMaskedLM were not initialized from the model checkpoint at raquiba/sarcasm-detection-BanglaSARC and are newly initialized: ['generator_lm_head.bias', 'generator_predictions.LayerNorm.bias', 'generator_predictions.LayerNorm.weight', 'generator_predictions.dense.bias', 'generator_predictions.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Total Pretrained tokens:  32000
Total number of tokens after adding the new ones:  32002


## Creating Custom Data Loader

In [6]:
class BertDataset(Dataset):
    def __init__(self, data, tokenizer,max_length):
        super(BertDataset, self).__init__()

        self.train_csv= data
        self.tokenizer=tokenizer
        self.target=self.train_csv.iloc[:,2]
        self.max_length=max_length

    def __len__(self):
        return len(self.train_csv)

    def __getitem__(self, index):

        text1 = self.train_csv.iloc[index,0] #0th column male text
        text2 = self.train_csv.iloc[index,1] #1st column female text

        inputs = self.tokenizer.batch_encode_plus(
            [text1, text2] ,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            max_length=self.max_length,
            truncation = True
        )
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        masks = inputs["attention_mask"]

        return torch.tensor(ids[0], dtype=torch.long), torch.tensor(token_type_ids[0], dtype=torch.long), torch.tensor(masks[0], dtype=torch.long), torch.tensor(ids[1], dtype=torch.long), torch.tensor(token_type_ids[1], dtype=torch.long), torch.tensor(masks[1], dtype=torch.long), torch.tensor(self.train_csv.iloc[index, 2], dtype=torch.long)

BATCH_SIZE = 4

dataset_train= BertDataset(train, tokenizer, max_length=512 )
dataset_val= BertDataset( validation, tokenizer, max_length=512)
dataset_test= BertDataset(test, tokenizer, max_length=512)
train_loader=DataLoader(dataset=dataset_train,batch_size=BATCH_SIZE, shuffle=True)
validation_loader=DataLoader(dataset=dataset_val,batch_size=BATCH_SIZE, shuffle=False)
test_loader=DataLoader(dataset=dataset_test,batch_size=BATCH_SIZE, shuffle=False)

## Creating Custom Model

In [7]:
import numpy as np
np.random.seed()
class BERT(nn.Module):
    def __init__(self, model):
        super(BERT, self).__init__()
        self.bert_model = model
        # print(self.bert_model.bert.embeddings.word_embeddings.weight)
        self.dropout = nn.Dropout(0.2)
        self.out = nn.Linear(768, 2)

    def forward(self,ids1, mask1, token_type_ids1, ids2, mask2, token_type_ids2, mode):
        if mode == 'training':
          ids = torch.vstack((ids1, ids2))
          mask = torch.vstack((mask1, mask2))
          token_type_ids = torch.vstack((token_type_ids1, token_type_ids2))
          o2= self.bert_model(ids,mask,token_type_ids).hidden_states[0]
          o2= o2.max(dim=1).values
          male_embedding, female_embedding = torch.vsplit(o2, 2)
          # print(original_embedding.shape, ner_embedding.shape)
          o2_male = self.dropout(male_embedding)
          out_male= self.out(o2_male)

          o2_female = self.dropout(female_embedding)
          out_female= self.out(o2_female)


          
          

          return (out_male + out_female)/2, F.softmax(out_male, dim=-1), F.softmax(out_female, dim=-1)


        if mode == 'inference':
          o2= self.bert_model(ids1, mask1, token_type_ids1).hidden_states[0]
          oe= o2.max(dim=1).values
          o2 = self.dropout(oe)
          out= self.out(o2)
          return out, oe

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
masked_model = nn.DataParallel(BERT(b_model)).to(device)

In [10]:
#debug
i1, t1,m1,i2,t2,m2,l = next(iter(train_loader))
i1 = i1.to(device)
t1 = t1.to(device)
m1 = m1.to(device)
i2 = i2.to(device)
t2 = t2.to(device)
m2 = m2.to(device)
l = l.to(device)
print('Input Shape: ', i1.shape)

logits, male_softmax, female_softmax = masked_model(i1, t1,m1,i2,t2,m2, 'training')
print(logits.shape, male_softmax.shape, male_softmax.shape)
logits, embedding = masked_model(i1, t1,m1,None,None,None, 'inference')
print(logits.shape, embedding.shape)

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2700: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Input Shape:  torch.Size([4, 512])
torch.Size([4, 2]) torch.Size([4, 2]) torch.Size([4, 2])
torch.Size([4, 2]) torch.Size([4, 768])


In [11]:
logits, logits.max(1)[1].view(-1,1)

(tensor([[ 0.4961, -0.0383],
         [ 0.4619, -0.2444],
         [ 0.0544, -0.0530],
         [ 0.4041,  0.2448]], device='cuda:0', grad_fn=<AddmmBackward0>),
 tensor([[0],
         [0],
         [0],
         [0]], device='cuda:0'))

In [12]:
logits.max(1)[1].view(-1,1).eq(l.view(-1,1)).sum().item()

1

## Creating Utility Class for Tracking Training Stat

In [8]:
from collections import OrderedDict
from collections import namedtuple
from itertools import product
import json
from torch.utils.tensorboard import SummaryWriter
import time
import torchvision
import pandas as pd
import torch
from IPython.display import display
from IPython.display import clear_output

class RunBuilder():
    @staticmethod
    def get_runs(params):
        Run = namedtuple('Run', params.keys())


        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))


        return runs


class RunManager():
    def __init__(self):
        self.epoch_count = 0
        self.epoch_ce_loss = 0
        self.epoch_kl_loss = 0
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_start_time = None

        self.epoch_valid_ce_loss = 0.0
        self.epoch_valid_kl_loss = 0.0
        self.epoch_valid_loss = 0.0
        self.epoch_num_valid_correct = 0

        self.run_params = None
        self.run_count = 0
        self.run_data = []
        self.run_start_time = None

        self.network = None
        self.loader = None
        self.validation_loader = None
        self.tb = None

    def begin_run(self, run, network, loader, validation_loader):
        self.run_start_time = time.time()

        self.run_params = run
        self.run_count += 1

        self.network = network
        self.loader = loader
        self.validation_loader = validation_loader
        self.tb = SummaryWriter(comment=f'-{run}')



    def end_run(self):
        self.tb.close()
        self.epoch_count = 0

    def begin_epoch(self):
        self.epoch_start_time = time.time()
        self.epoch_count += 1
        self.epoch_loss = 0
        self.epoch_ce_loss = 0
        self.epoch_kl_loss = 0
        self.epoch_num_correct = 0
        self.epoch_valid_ce_loss = 0
        self.epoch_valid_kl_loss = 0
        self.epoch_valid_loss = 0
        self.epoch_num_valid_correct = 0

    def end_epoch(self):
        epoch_duration = time.time() - self.epoch_start_time
        run_duration = time.time() - self.run_start_time

        ce_loss = self.epoch_ce_loss/len(self.loader.dataset)
        kl_loss = self.epoch_kl_loss/len(self.loader.dataset)
        loss = self.epoch_loss/len(self.loader.dataset)
        accuracy = self.epoch_num_correct/len(self.loader.dataset)
        val_ce_loss = self.epoch_valid_ce_loss/len(self.validation_loader.dataset)
        val_kl_loss = self.epoch_valid_kl_loss/len(self.validation_loader.dataset)
        val_loss = self.epoch_valid_loss/len(self.validation_loader.dataset)
        val_accuracy = self.epoch_num_valid_correct/len(self.validation_loader.dataset)

        self.tb.add_scalar('CE Loss', ce_loss, self.epoch_count)
        self.tb.add_scalar('KL Loss', kl_loss, self.epoch_count)
        self.tb.add_scalar('Loss', loss, self.epoch_count)
        self.tb.add_scalar('Accuracy', accuracy, self.epoch_count)
        self.tb.add_scalar('Validation CE Loss', val_ce_loss, self.epoch_count)
        self.tb.add_scalar('Validation KL Loss', val_kl_loss, self.epoch_count)
        self.tb.add_scalar('Validation Loss', val_loss, self.epoch_count)
        self.tb.add_scalar('Validation Accuracy', val_accuracy, self.epoch_count)
        print('-------------')
        for name, param in self.network.named_parameters():
            # print(name, param, self.epoch_count)
            self.tb.add_histogram(name, param, self.epoch_count)

        results = OrderedDict()
        results["run"] = self.run_count
        results["epoch"] = self.epoch_count
        results["ce_loss"] = ce_loss
        results["kl_loss"] = kl_loss
        results["loss"] = loss
        results["accuracy"] = accuracy
        results["validation ce loss"] = val_ce_loss
        results["validation kl loss"] = val_kl_loss
        results["validation loss"] = val_loss
        results["validation accuracy"] = val_accuracy
        results["epoch duration (minutes)"] = epoch_duration/60
        results["run duration (minutes)"] = run_duration/60
        for k,v in self.run_params._asdict().items(): results[k] = v
        self.run_data.append(results)
        df = pd.DataFrame.from_dict(self.run_data, orient='columns')

        clear_output(wait=True)
        display(df)

    def track_loss(self, ce_loss, kl_loss, loss):
        self.epoch_ce_loss += ce_loss.item()#*self.loader.batch_size
        self.epoch_kl_loss += kl_loss.item()#*self.loader.batch_size
        self.epoch_loss += loss.item()#*self.loader.batch_size

    def track_num_correct(self, preds, labels):
        self.epoch_num_correct += self._get_num_correct(preds, labels)

    def track_validation_loss(self, ce_loss, kl_loss, loss):
        self.epoch_valid_ce_loss += ce_loss.item()#*self.loader.batch_size
        self.epoch_valid_kl_loss += kl_loss.item()#*self.loader.batch_size
        self.epoch_valid_loss += loss.item()#*self.loader.batch_size

    def track_num_validation_correct(self, preds, labels):
        self.epoch_num_valid_correct += self._get_num_correct(preds, labels)

    @torch.no_grad()
    def _get_num_correct(self, preds, labels):
        return preds.max(1)[1].view(-1,1).eq(labels).sum().item()

    def save(self, fileName):
        pd.DataFrame.from_dict(
            self.run_data,
            orient='columns').to_csv(rf'{fileName}.csv')

## Creating Training Loop

In [9]:
SAVE_MODEL = True
def my_trainer(train_loader, validation_loader, network, LR = 1e-5 ,epochs = 5, BATCH_SIZE = 16,  fresh_training = True, validation = True):
    params = OrderedDict(
        lr = [LR],
        batch_size =[BATCH_SIZE],
        device = ['cuda' if torch.cuda.is_available() else 'cpu'],
    )

    m = RunManager()
    for run in RunBuilder.get_runs(params):
        device = torch.device(run.device)
        # network = network.to(device)
        # optimizer = optim.Adam(network.parameters(), lr=run.lr)

        loss_fn = nn.CrossEntropyLoss()
        loss_fn_2 = nn.KLDivLoss(reduction="batchmean")

        total_steps = len(train_loader) * epochs
        optimizer = optim.Adam(network.parameters(), lr= run.lr)

        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)
        if not fresh_training:
            load_model(network, optimizer, PATH = 'Sarcasm_KLDM_Average.pth')
        m.begin_run(run, network, train_loader, validation_loader)


        for epoch in range(epochs):
            network.train(True)
            print(f'Epoch {epoch + 1} : ', end = '\t')
            m.begin_epoch()


            for batch in tqdm(train_loader):
                input_ids1, token_type_ids1, attention_masks1, input_ids2, token_type_ids2, attention_masks2, labels = batch
                input_ids1 = input_ids1.to(device)
                token_type_ids1 = token_type_ids1.to(device)
                attention_masks1 = attention_masks1.to(device)

                input_ids2 = input_ids2.to(device)
                token_type_ids2 = token_type_ids2.to(device)
                attention_masks2 = attention_masks2.to(device)

                labels = labels.type(torch.LongTensor)
                labels = labels.to(device)
                copy_labels = labels

                preds, male_softmax, female_softmax = network(input_ids1, attention_masks1, token_type_ids1, input_ids2, attention_masks2, token_type_ids2, 'training')
                labels = labels.view(-1, 1)



                loss1 = loss_fn(preds, copy_labels)
                loss2 = loss_fn_2(male_softmax.log(), female_softmax) + loss_fn_2(female_softmax.log(), male_softmax)
                loss = loss1 + loss2
                # print(preds.shape, labels.shape)
                # print(preds, labels)


                optimizer.zero_grad()
                loss.backward()

                torch.nn.utils.clip_grad_norm_(network.parameters(), 1.0)

                optimizer.step()

                scheduler.step()


                m.track_loss(loss1, loss2, loss)
                m.track_num_correct(preds, labels)


            network.eval()
            # print(network.bert_model.bert.embeddings.word_embeddings.weight)
            for batch in tqdm(validation_loader):
                input_ids1, token_type_ids1, attention_masks1, input_ids2, token_type_ids2, attention_masks2, labels = batch
                input_ids1 = input_ids1.to(device)
                token_type_ids1 = token_type_ids1.to(device)
                attention_masks1 = attention_masks1.to(device)
                input_ids2 = input_ids2.to(device)
                token_type_ids2 = token_type_ids2.to(device)
                attention_masks2 = attention_masks2.to(device)

                labels = labels.type(torch.LongTensor)
                labels = labels.to(device)
                copy_labels = labels
                with torch.no_grad():
                    preds, male_softmax, female_softmax = network(input_ids1, attention_masks1, token_type_ids1, input_ids2, attention_masks2, token_type_ids2, 'training')
                    labels = labels.view(-1, 1)
                    loss1 = loss_fn(preds, copy_labels)
                    loss2 = loss_fn_2(male_softmax.log(), female_softmax) + loss_fn_2(female_softmax.log(),male_softmax)
                    loss = loss1 + loss2
                    m.track_validation_loss(loss1, loss2, loss)
                    m.track_num_validation_correct(preds, labels)
                # break #not doing validation here




            m.end_epoch()
            network.train(False)
            save_model(network, optimizer)
        m.end_run()
    if SAVE_MODEL == True:
        save_model(network, optimizer)

    s = datetime.now().strftime("%d-%m-%Y %I:%M:%p").replace(" ", "_").replace(":", "_")
    PATH  = rf'results_{s}'
    m.save(PATH)
    return


def save_model(network, optimizer):
    s = datetime.now().strftime("%d-%m-%Y %I:%M:%p").replace(" ", "_").replace(":", "_")
    PATH  = rf'Sarcasm_KLDM_Average.pth'
    torch.save({
    'model_state_dict': network.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    }, PATH)
    print(f'Saved model and optimizer at {PATH}')
    return

def load_model(network, optimizer, PATH ):
    checkpoint = torch.load(PATH)
    network.load_state_dict(checkpoint['model_state_dict'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print(rf'Loaded model and optimizer from {PATH}')
    else:
        print(rf'Loaded model from {PATH}')

    return

def get_parameter_info(network, details = False):
    print('Total parameters:' ,(sum(p.numel() for p in network.parameters())))
    print('Total trainable parameters:' ,(sum(p.numel() for p in network.parameters() if p.requires_grad)))
    if details:
        print('---------------------------------\nDetailed Parameter Info: ')

        for name, parameter in network.named_parameters():
            print('name, parameter.requires_grad, parameter.is_cuda, parameter.size(): ', name, parameter.requires_grad, parameter.is_cuda, parameter.size())
            print('---------------------------------')
    return

if finetuning:
  my_trainer(train_loader, validation_loader, masked_model, 0.0001, epochs = 15, BATCH_SIZE = BATCH_SIZE, fresh_training = True)

,run,epoch,ce_loss,kl_loss,loss,accuracy,validation ce loss,validation kl loss,validation loss,validation accuracy,epoch duration (minutes),run duration (minutes),lr,batch_size,device
0,1,1,0.164848,0.012807,0.177655,0.603356,0.153539,0.000862,0.154401,0.729730,1.831621,1.831655,0.0001,4,cuda
1,1,2,0.125441,0.010517,0.135958,0.802286,0.121166,0.001732,0.122898,0.777027,1.830701,3.710696,0.0001,4,cuda
2,1,3,0.087264,0.014180,0.101444,0.871231,0.104383,0.002672,0.107055,0.790541,1.819233,5.578943,0.0001,4,cuda
3,1,4,0.062687,0.015845,0.078532,0.912695,0.091893,0.003859,0.095752,0.851351,1.826487,7.452812,0.0001,4,cuda
4,1,5,0.046227,0.016814,0.063041,0.934339,0.086654,0.004141,0.090795,0.831081,1.834637,9.334987,0.0001,4,cuda
5,1,6,0.035605,0.016131,0.051735,0.949052,0.083510,0.004270,0.087780,0.871622,1.842108,11.223125,0.0001,4,cuda
6,1,7,0.028583,0.015523,0.044106,0.958171,0.086251,0.004178,0.090428,0.891892,1.844126,13.114487,0.0001,4,cuda
7,1,8,0.022601,0.013573,0.036174,0.966075,0.085679,0.004199,0.089879,0.878378,1.844053,15.004724,0.0001,4,cuda
8,1,9,0.019274,0.012826,0.032100,0.970939,0.091556,0.003396,0.094952,0.885135,1.827426,16.878322,0.0001,4,cuda
9,1,10,0.015386,0.012439,0.027825,0.977626,0.090444,0.003387,0.093831,0.878378,1.824394,18.748790,0.0001,4,cuda


Saved model and optimizer at Sarcasm_KLDM_Average.pth
Saved model and optimizer at Sarcasm_KLDM_Average.pth


## Load Saved Model

In [10]:
#Load Model
from tqdm import tqdm
import torch
import torch.optim as optim
import torch.nn.functional as F
from collections import OrderedDict
from datetime import datetime
from transformers import get_linear_schedule_with_warmup

def load_model(network, optimizer, PATH ):
    checkpoint = torch.load(PATH, map_location=torch.device('cpu'))
    network.load_state_dict(checkpoint['model_state_dict'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print(rf'Loaded model and optimizer from {PATH}')
    else:
        print(rf'Loaded model from {PATH}')

    return

# load_and_run = 1
if load_and_run:
#   load_model(masked_model, None, PATH = '/kaggle/input/toxicity_kldm/pytorch/default/1/Toxicity_KLDM.pth')
  load_model(masked_model, None, PATH = 'Sarcasm_KLDM_Average.pth')

## Inference

In [11]:
import random
import warnings
warnings.filterwarnings('ignore')
def get_prediction(data_loader, network, choice = 'original'):
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  device = torch.device(device)
  network = network.to(device)
  network.eval()
  l = []
  emb = []
  for batch in tqdm(data_loader):
    input_ids1, token_type_ids1, attention_masks1, input_ids2, token_type_ids2, attention_masks2, labels = batch
    input_ids1 = input_ids1.to(device)
    token_type_ids1 = token_type_ids1.to(device)
    attention_masks1 = attention_masks1.to(device)
    input_ids2 = input_ids2.to(device)
    token_type_ids2 = token_type_ids2.to(device)
    attention_masks2 = attention_masks2.to(device)
    labels = labels.type(torch.LongTensor)
    labels = labels.to(device)
    with torch.no_grad():
        if choice == 'male':
          preds, hidden_states = network(input_ids1, attention_masks1,token_type_ids1, None, None, None, 'inference')
        else:
          preds, hidden_states = network(input_ids2, attention_masks2,token_type_ids2, None, None, None, 'inference')

        labels = labels.type_as(preds).view(-1, 1)
        preds =  preds.max(1)[1].view(-1,1)
        l.append(preds)
        emb.append(hidden_states)
  return l, torch.vstack(emb).cpu().numpy()

p, emb_male = get_prediction(test_loader, masked_model, 'male')
test['pred_male_kldm'] = [x[0] for pred in p for x in pred.tolist()]
p, emb_male = get_prediction(test_loader, masked_model, 'female')
test['pred_female_kldm'] = [x[0] for pred in p for x in pred.tolist()]
x = sum(test['pred_male_kldm']!=test['_original_label'])
print('For Male:\nTotal test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1- (x/len(test))) 
x = sum(test['pred_female_kldm']!=test['_original_label'])
print('For Female:\nTotal test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1- (x/len(test)))
print('Male Female Mismatch: ', sum(test['pred_male_kldm']!=test['pred_female_kldm']))

100%|██████████| 463/463 [00:09<00:00, 50.18it/s]

For Male:
Total test data:  1850 Total mismatch:  199 Accuracy:  0.8924324324324324
For Female:
Total test data:  1850 Total mismatch:  210 Accuracy:  0.8864864864864865
Male Female Mismatch:  33


# KLD Minimization Result

In [12]:
import numpy as np
np.random.seed(0)

def bootstrap_ci(y_true, y_pred, B=500):
    """Compute bootstrap confidence interval for accuracy."""
    N = len(y_true)
    original_acc = np.mean(y_pred == y_true)

    bootstrap_accs = []

    for _ in range(B):
        indices = np.random.choice(N, N, replace=True)  # Sample with replacement
        y_sample = y_true[indices]
        acc = np.mean(y_pred[indices] == y_sample)
        bootstrap_accs.append(acc)
    # Compute 95% confidence interval
    lower, upper = np.percentile(bootstrap_accs, [2.5, 97.5])

    return original_acc, (lower, upper), np.mean(bootstrap_accs)

y_true = test['_original_label'].values  # Ground truth labels
y_pred = test['pred_male_kldm'].values
acc, ci, ma = bootstrap_ci(y_true, y_pred)
print('Male: ')
print(f"Accuracy: {acc*100:.2f}")
print(f"95% Confidence Interval: {ci[0]*100:.2f}  {ci[1]*100:.2f}")
print(f"mean accuracy: {ma*100: .2f}")

y_true = test['_original_label'].values
y_pred = test['pred_female_kldm'].values
acc, ci, ma = bootstrap_ci(y_true, y_pred)
print('\nFemale: ')
print(f"Accuracy: {acc*100:.2f}")
print(f"95% Confidence Interval: {ci[0]*100:.2f}  {ci[1]*100:.2f}")
print(f"mean accuracy: {ma*100: .2f}")
print('Male Female Mismatch: ', sum(test['pred_male_kldm']!=test['pred_female_kldm']))
test.to_csv('revision_kldm_average_sarcasm_result.csv', index = False)
print('Saved....')

Male: 
Accuracy: 89.24
95% Confidence Interval: 87.78  90.62
mean accuracy:  89.23

Female: 
Accuracy: 88.65
95% Confidence Interval: 87.27  90.08
mean accuracy:  88.64
Male Female Mismatch:  33
Saved....


In [13]:
# Statistical Parity Difference (SPD) and Equal Opportunity Difference (EOD) Calculation
import numpy as np
import pandas as pd
np.random.seed(0)

test = pd.read_csv('revision_kldm_average_sarcasm_result.csv')


# Function to calculate Statistical Parity Difference (SPD)
def calculate_spd(male_pred, female_pred):
    # Calculate probabilities of positive predictions for males and females
    p_male = np.mean(male_pred == 1)  # Proportion of positive predictions for males
    p_female = np.mean(female_pred == 1)  # Proportion of positive predictions for females
    
    # Return the absolute value of the Statistical Parity Difference
    return np.abs(p_male - p_female)

# Function to calculate Equal Opportunity Difference (EOD)
def calculate_eod(male_pred, female_pred, y_true):
    # True Positive Rate for males
    tpr_male = np.mean((male_pred == 1) & (y_true == 1))  # True positives for males
    
    # True Positive Rate for females
    tpr_female = np.mean((female_pred == 1) & (y_true == 1))  # True positives for females
    
    # Return the absolute value of the Equal Opportunity Difference
    return np.abs(tpr_male - tpr_female)

# Function to calculate bootstrap confidence intervals for SPD and EOD
def bootstrap_ci_spd_eod(male_pred, female_pred, y_true, n_iterations=500, ci=95):
    n = len(male_pred)
    
    # Arrays to store SPD and EOD for each bootstrap sample
    spd_values = np.zeros(n_iterations)
    eod_values = np.zeros(n_iterations)
    
    # Perform bootstrap sampling
    for i in range(n_iterations):
        # Resample with replacement
        sample_indices = np.random.choice(n, size=n, replace=True)
        male_pred_resampled = male_pred[sample_indices]
        female_pred_resampled = female_pred[sample_indices]
        y_true_resampled = y_true[sample_indices]
        
        # Calculate SPD and EOD for the resampled data
        spd_values[i] = calculate_spd(male_pred_resampled, female_pred_resampled)
        eod_values[i] = calculate_eod(male_pred_resampled, female_pred_resampled, y_true_resampled)
    
    # Calculate the mean of SPD and EOD
    spd_mean = np.mean(spd_values)
    eod_mean = np.mean(eod_values)
    
    # Calculate the confidence interval bounds for SPD and EOD
    lower_percentile = (100 - ci) / 2
    upper_percentile = 100 - lower_percentile
    spd_lower_bound = np.percentile(spd_values, lower_percentile)
    spd_upper_bound = np.percentile(spd_values, upper_percentile)
    eod_lower_bound = np.percentile(eod_values, lower_percentile)
    eod_upper_bound = np.percentile(eod_values, upper_percentile)
    
    return (np.abs(spd_mean), np.abs(spd_lower_bound), np.abs(spd_upper_bound)), (np.abs(eod_mean), np.abs(eod_lower_bound), np.abs(eod_upper_bound))


y_true = test._original_label.values
male_pred = test.pred_male_kldm.values
female_pred = test.pred_female_kldm.values

# Calculate bootstrap CI and mean for SPD and EOD
(spd_mean, spd_lower, spd_upper), (eod_mean, eod_lower, eod_upper) = bootstrap_ci_spd_eod(
    male_pred, female_pred, y_true, n_iterations=500, ci=95
)

print(f"SPD 95% Confidence Interval: [{spd_lower:.3f}, {spd_upper:.3f}]\tSPD Mean: {spd_mean:.3f}")
print('---------------------------------------------------------------------------------------------')
print(f"EOD 95% Confidence Interval: [{eod_lower:.3f}, {eod_upper:.3f}]\tEOD Mean: {eod_mean:.3f}")

SPD 95% Confidence Interval: [0.000, 0.007]	SPD Mean: 0.002
---------------------------------------------------------------------------------------------
EOD 95% Confidence Interval: [0.000, 0.008]	EOD Mean: 0.003


In [ ]:
# Male: 
# Accuracy: 0.9043
# 95% Confidence Interval: (np.float64(0.8882801418439716), np.float64(0.9198936170212766))
# mean accuracy:  90.39

# Female: 
# Accuracy: 0.8950
# 95% Confidence Interval: (np.float64(0.8797695035460993), np.float64(0.9113475177304965))
# mean accuracy:  89.50
# Male Female Mismatch:  33
# Saved....

In [ ]:
# Male: 
# Accuracy: 0.9071
# 95% Confidence Interval: (0.8918262411347517, 0.9223581560283688)
# mean accuracy: 0.9068921985815602

# Female: 
# Accuracy: 0.8965
# 95% Confidence Interval: (0.8818971631205674, 0.9113475177304965)
# mean accuracy: 0.896463829787234
# Male Female Mismatch:  43
# Saved....

## Result of Approach 2

In [ ]:
x = sum(test['original_lebel'] != test['a2_original_pred'])
print('For Original text:\nTotal test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)) )
x = sum(test['original_lebel'] != test['a2_ner_pred'])
print('For NER Converted text:\nTotal test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)))
print('Bias: ', sum(test['a2_ner_pred'] != test['a2_original_pred']))

In [ ]:
x = sum(test['original_lebel'] != test['a2_original_pred'])
print('For Original text:\nTotal test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)) )
x = sum(test['original_lebel'] != test['a2_ner_pred'])
print('For NER Converted text:\nTotal test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)))
print('Bias: ', sum(test['a2_ner_pred'] != test['a2_original_pred']))

In [ ]:
x = test[test['a2_ner_pred'] != test['a2_original_pred']][['Original Sentence', 'Converted Sentence', 'original_lebel', 'a2_original_pred', 'a2_ner_pred']]
print(len(x))
x

## Sentence Embedding Visualization for Original Sentence and NER Converted Sentence

### For Training Data

In [ ]:
#for visualizing training data embedding
dataset_train= BertDataset(train, tokenizer, max_length=200)
train_loader=DataLoader(dataset=dataset_train,batch_size=16, shuffle = False)
p, emb_ori = get_prediction(train_loader, masked_model, 'original')
p, emb_ner = get_prediction(train_loader, masked_model, 'ner')

In [ ]:
from sklearn.manifold import TSNE
import plotly.express as px

tsne_cls = TSNE(n_components=2, random_state=42).fit_transform(emb_ori)

labels = train['original_lebel']   # Extract labels
review_texts = train['Original Sentence']  # Extract review texts

# Prepare the data for Plotly plots
plot_data_cls = {'t-SNE Dimension 1': tsne_cls[:, 0],
                 't-SNE Dimension 2': tsne_cls[:, 1],
                 'Label': labels,
                 'Text': review_texts}

# Customize colorbar to show integer labels
colorbar_tickvals = list(range(1, 6))
colorbar_ticktext = [str(val) for val in colorbar_tickvals]

# Plotting with Plotly - [CLS] Token Embeddings
fig_cls = px.scatter(plot_data_cls, x='t-SNE Dimension 1', y='t-SNE Dimension 2',
                     color='Label', hover_data={'t-SNE Dimension 1': False, 't-SNE Dimension 2': False, 'Text': True})


fig_cls.update_layout(title='t-SNE of Embeddings Original Training Data with Labels', height=700,  font=dict(
        size=20,
))
fig_cls.update_traces(marker=dict(size=10, opacity=0.9))

fig_cls.update_layout(coloraxis_colorbar=dict(
    tickvals=colorbar_tickvals,
    ticktext=colorbar_ticktext
))

fig_cls.show()


In [ ]:
from sklearn.manifold import TSNE
import plotly.express as px

tsne_cls = TSNE(n_components=2, random_state=42).fit_transform(emb_ner)

labels = train['original_lebel']   # Extract labels
review_texts = train['Converted Sentence']  # Extract review texts

# Prepare the data for Plotly plots
plot_data_cls = {'t-SNE Dimension 1': tsne_cls[:, 0],
                 't-SNE Dimension 2': tsne_cls[:, 1],
                 'Label': labels,
                 'Text': review_texts}

# Customize colorbar to show integer labels
colorbar_tickvals = list(range(1, 6))
colorbar_ticktext = [str(val) for val in colorbar_tickvals]

# Plotting with Plotly - [CLS] Token Embeddings
fig_cls = px.scatter(plot_data_cls, x='t-SNE Dimension 1', y='t-SNE Dimension 2',
                     color='Label', hover_data={'t-SNE Dimension 1': False, 't-SNE Dimension 2': False, 'Text': True})


fig_cls.update_layout(title='t-SNE of Embeddings NER Converted Training Data with Labels', height=700,  font=dict(
        size=20,
))
fig_cls.update_traces(marker=dict(size=10, opacity=0.9))

fig_cls.update_layout(coloraxis_colorbar=dict(
    tickvals=colorbar_tickvals,
    ticktext=colorbar_ticktext
))

fig_cls.show()


### For Biased Test Data

In [ ]:
#for visualizing training data embedding
dataset_train= BertDataset(x, tokenizer, max_length=200)
train_loader=DataLoader(dataset=dataset_train,batch_size=16, shuffle = False)
p, emb_ori = get_prediction(train_loader, masked_model, 'original')
p, emb_ner = get_prediction(train_loader, masked_model, 'ner')

In [ ]:
from sklearn.manifold import TSNE
import plotly.express as px

tsne_cls = TSNE(n_components=2, random_state=42).fit_transform(emb_ori)

labels = x['original_lebel']   # Extract labels
review_texts = x['Original Sentence']  # Extract review texts

# Prepare the data for Plotly plots
plot_data_cls = {'t-SNE Dimension 1': tsne_cls[:, 0],
                 't-SNE Dimension 2': tsne_cls[:, 1],
                 'Label': labels,
                 'Text': review_texts}

# Customize colorbar to show integer labels
colorbar_tickvals = list(range(1, 6))
colorbar_ticktext = [str(val) for val in colorbar_tickvals]

# Plotting with Plotly - [CLS] Token Embeddings
fig_cls = px.scatter(plot_data_cls, x='t-SNE Dimension 1', y='t-SNE Dimension 2',
                     color='Label', hover_data={'t-SNE Dimension 1': False, 't-SNE Dimension 2': False, 'Text': True})


fig_cls.update_layout(title='t-SNE of Embeddings Original Training Data with Labels', height=700,  font=dict(
        size=20,
))
fig_cls.update_traces(marker=dict(size=10, opacity=0.9))

fig_cls.update_layout(coloraxis_colorbar=dict(
    tickvals=colorbar_tickvals,
    ticktext=colorbar_ticktext
))

fig_cls.show()

In [ ]:
from sklearn.manifold import TSNE
import plotly.express as px

tsne_cls = TSNE(n_components=2, random_state=42).fit_transform(emb_ner)

labels = x['original_lebel']   # Extract labels
review_texts = x['Converted Sentence']  # Extract review texts

# Prepare the data for Plotly plots
plot_data_cls = {'t-SNE Dimension 1': tsne_cls[:, 0],
                 't-SNE Dimension 2': tsne_cls[:, 1],
                 'Label': labels,
                 'Text': review_texts}

# Customize colorbar to show integer labels
colorbar_tickvals = list(range(1, 6))
colorbar_ticktext = [str(val) for val in colorbar_tickvals]

# Plotting with Plotly - [CLS] Token Embeddings
fig_cls = px.scatter(plot_data_cls, x='t-SNE Dimension 1', y='t-SNE Dimension 2',
                     color='Label', hover_data={'t-SNE Dimension 1': False, 't-SNE Dimension 2': False, 'Text': True})


fig_cls.update_layout(title='t-SNE of Embeddings NER Converted Training Data with Labels', height=700,  font=dict(
        size=20,
))
fig_cls.update_traces(marker=dict(size=10, opacity=0.9))

fig_cls.update_layout(coloraxis_colorbar=dict(
    tickvals=colorbar_tickvals,
    ticktext=colorbar_ticktext
))

fig_cls.show()

In [ ]:
test.to_csv('/content/drive/MyDrive/#Research/# GB/toxicity_reSult_a2.csv', index = False)